# Use scikit-learn to predict diabetes progression with Watson Machine Learning REST API

This notebook contains steps and code to demonstrate support of external machine learning models in Watson Machine Learning Service. This notebook introduces `cURL` calls for publishing, deploying (Web Service) and scoring ML scikit model.

Some familiarity with cURL is helpful. This notebook uses cURL examples.

This example was made based on `diabetes` dataset and trained scikit ML model that could be found inside the same repository under as follows: 
- `/data/diabetes/`
- `/models/scikit/diabetes/`


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning repository, deployment and scoring.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)  
2.	[WML Repository](#wml_repository)   
3.	[Model Deployment and Scoring](#deploy_and_score)   
4.  [Persist new version of the model](#update_model)
5.  [Redeploy and score new version of the model](#redeploy)
6.	[Cleaning](#cleaning)  
7.	[Summary](#summary)  

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `password`.

In [48]:
%env USERNAME=
%env PASSWORD=
%env DATAPLATFORM_URL=

%env SPACE_ID=

env: USERNAME=
env: PASSWORD=
env: DATAPLATFORM_URL=
env: SPACE_ID=


<a id="wml_token"></a>
### Getting WML authorization token for further cURL calls

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" target="_blank" rel="noopener no referrer">Example of cURL call to get WML token</a>

In [2]:
%%bash --out token

curl -sk -X GET \
    --user $USERNAME:$PASSWORD \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/v1/preauth/validateAuth" \
    | cut -d '"' -f 44

In [47]:
%env TOKEN=$token

<a id="space_creation"></a>
### Space creation
**Tip:** If you do not have `space` already created, please convert below three cells to `code` and run them.

First of all, you need to create a `space` that will be used in all of your further cURL calls. 
If you do not have `space` already created, below is the cURL call to create one.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_create" 
target="_blank" rel="noopener no referrer">Space creation</a>

Space creation is asynchronous. This means that you need to check space creation status after creation call.
Make sure that your newly created space is `active`.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_get" 
target="_blank" rel="noopener no referrer">Get space information</a>

---

<a id="wml_repository"></a>
## 2. Manage WML Repository

In this section you will learn how to upload your ML model, list your models stored in repository, delete your model and update it.

<a id="model_storing"></a>
### Model storing
Store information about your model to WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create" 
target="_blank" rel="noopener no referrer">Model storing</a>

In [4]:
%%bash --out model_payload

MODEL_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "scikit_diabetes_model","description": "This is description","type": "scikit-learn_0.20", "software_spec": {"name": "scikit-learn_0.20-py3.6"}}'
echo $MODEL_PAYLOAD | python -m json.tool

In [5]:
%env MODEL_PAYLOAD=$model_payload

env: MODEL_PAYLOAD={
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "name": "scikit_diabetes_model",
    "description": "This is description",
    "type": "scikit-learn_0.20",
    "software_spec": {
        "name": "scikit-learn_0.20-py3.6"
    }
}


In [6]:
%%bash --out model_id -s "$model_payload"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [7]:
%env MODEL_ID=$model_id

env: MODEL_ID=c8c520cb-7298-4d30-a338-db62cdda125b


### Create model revision for further update

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create_revision" 
target="_blank" rel="noopener no referrer">Model revision</a>

In [8]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Initial model."}'
echo $REVISION_PAYLOAD | python -m json.tool

In [9]:
%env REVISION_PAYLOAD=$revision_payload

env: REVISION_PAYLOAD={
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "commit_message": "Initial model."
}


In [10]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "software_spec": {
            "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
            "name": "scikit-learn_0.20-py3.6"
        },
        "type": "scikit-learn_0.20"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Initial model.",
            "committed_at": "2020-10-01T11:02:09.002Z"
        },
        "created_at": "2020-10-01T11:00:54.535Z",
        "description": "This is description",
        "id": "c8c520cb-7298-4d30-a338-db62cdda125b",
        "modified_at": "2020-10-01T11:00:54.535Z",
        "name": "scikit_diabetes_model",
        "owner": "1000330999",
        "rev": "1",
        "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a"
    }
}


<a id="content_upload"></a>
### Model content upload
Now you need to upload your model content into the WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_upload_content" 
target="_blank" rel="noopener no referrer">Upload model content</a>

In [15]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@../../../../../../models/scikit/diabetes/model/diabetes_model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | grep "attachment_id" | awk -F '"' '{ print $4 }'

In [16]:
%env ATTACHMENT_ID=$attachment_id

env: ATTACHMENT_ID=e5cb47cf-9760-4a17-bf64-eec02eb5a6db


<a id="model_download"></a>
### Download model
If you want to download your saved model, please make the following call.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_filtered_download" 
target="_blank" rel="noopener no referrer">Download model content</a>

In [17]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [18]:
!ls -l model.tar.gz

-rw-r--r--  1 amadeusz.masny1@ibm.com  staff  765 Oct  1 13:12 model.tar.gz


<a id="deploy_and_score"></a>
## 3. Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

<a id="deployment_creation"></a>
### Deployment creation
This example uses scikit-learn model deployment and `S` hardware specification.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create" 
target="_blank" rel="noopener no referrer">Create deployment</a>

In [19]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Diabetes deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD | python -m json.tool 

In [20]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

env: DEPLOYMENT_PAYLOAD={
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "name": "Diabetes deployment",
    "description": "This is description",
    "online": {},
    "hardware_spec": {
        "name": "S"
    },
    "asset": {
        "id": "c8c520cb-7298-4d30-a338-db62cdda125b"
    }
}


In [21]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p

In [22]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=fda706ed-6eb7-476e-bbee-9f5a9e688aa0


<a id="deployment_details"></a>
### Get deployment details
As deployment API is asynchronous, please make sure your deployment is in `ready` state before going to the next points.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_get" 
target="_blank" rel="noopener no referrer">Get deployment details</a>

In [30]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "c8c520cb-7298-4d30-a338-db62cdda125b"
        },
        "custom": {},
        "deployed_asset_type": "model",
        "description": "This is description",
        "hardware_spec": {
            "id": "Not_Applicable",
            "name": "S",
            "num_nodes": 1
        },
        "name": "Diabetes deployment",
        "online": {},
        "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
        "status": {
            "online_url": {
                "url": "https://sep29b93-cpd-sep29b93.apps.ocwmlfvt454.cp.fyre.ibm.com/ml/v4/deployments/fda706ed-6eb7-476e-bbee-9f5a9e688aa0/predictions"
            },
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2020-10-01T11:13:04.822Z",
        "description": "This is description",
        "id": "fda706ed-6eb7-476e-bbee-9f5a9e688aa0",
        "modified_at": "2020-10-01T11:13:04.822Z",
        "name": "Diabetes deployment",
        "owner": "

<a id="webservice_score"></a>
### Scoring of a webservice
If you want to make a `score` call on your deployment, please follow a below method:

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_compute_predictions" 
target="_blank" rel="noopener no referrer">Score your deployment</a>

In [24]:
%%bash --out scoring_payload

SCORING_PAYLOAD='{"space_id": "$SPACE_ID","input_data": [{"fields": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"], "values": [[-0.00188201652779104, -0.044641636506989, -0.0514740612388061, -0.0263278347173518, -0.00844872411121698, -0.019163339748222, 0.0744115640787594, -0.0394933828740919, -0.0683297436244215, -0.09220404962683], [0.0852989062966783, 0.0506801187398187, 0.0444512133365941, -0.00567061055493425, -0.0455994512826475, -0.0341944659141195, -0.0323559322397657, -0.00259226199818282, 0.00286377051894013, -0.0259303389894746]]}]}'
echo $SCORING_PAYLOAD | python -m json.tool

In [25]:
%env SCORING_PAYLOAD=$scoring_payload

env: SCORING_PAYLOAD={
    "space_id": "$SPACE_ID",
    "input_data": [
        {
            "fields": [
                "age",
                "sex",
                "bmi",
                "bp",
                "s1",
                "s2",
                "s3",
                "s4",
                "s5",
                "s6"
            ],
            "values": [
                [
                    -0.00188201652779104,
                    -0.044641636506989,
                    -0.0514740612388061,
                    -0.0263278347173518,
                    -0.00844872411121698,
                    -0.019163339748222,
                    0.0744115640787594,
                    -0.0394933828740919,
                    -0.0683297436244215,
                    -0.09220404962683
                ],
                [
                    0.0852989062966783,
                    0.0506801187398187,
                    0.0444512133365941,
                    -0.00567061055493425,
          

In [31]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD"\
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2020-08-01" \
    | python -m json.tool

{
    "predictions": [
        {
            "fields": [
                "prediction"
            ],
            "values": [
                [
                    68.44064739029959
                ],
                [
                    178.20406005411206
                ]
            ]
        }
    ]
}


<a id="deployments_list"></a>
### Listing all deployments

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_list" 
target="_blank" rel="noopener no referrer">List deployments details</a>

In [32]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "resources": [
        {
            "entity": {
                "asset": {
                    "id": "3f6336a3-b20d-4fd1-9ec6-07797344c799"
                },
                "batch": {},
                "custom": {},
                "deployed_asset_type": "model",
                "description": "This is description",
                "hybrid_pipeline_hardware_specs": [
                    {
                        "hardware_spec": {
                            "name": "M"
                        },
                        "node_runtime_id": "auto_ai.kb"
                    }
                ],
                "name": "AutoAI deployment",
                "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
                "status": {
                    "state": "ready"
                }
            },
            "metadata": {
                "created_at": "2020-10-01T09:35:43.215Z",
                "description": "This is description",
                "id": "4f3ba600-bbe4-4b13-a

<a id="update_model"></a>
## 4. Persist new version of the model

In this section, you'll learn how to store new version of your model in Watson Machine Learning repository.

### Model update
Below you can find how ML model can be updated with new version on WML repository.

List model revisions.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_list_revisions" 
target="_blank" rel="noopener no referrer">List revisions</a>

In [33]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "first": {
        "href": "/ml/v4/models/c8c520cb-7298-4d30-a338-db62cdda125b/revisions?space_id=2219f598-6194-4e9a-9287-0730c69b5f4a&limit=100"
    },
    "limit": 100,
    "resources": [
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Initial model.",
                    "committed_at": "2020-10-01T11:02:09.002Z"
                },
                "created_at": "2020-10-01T11:00:54.535Z",
                "description": "This is description",
                "id": "c8c520cb-7298-4d30-a338-db62cdda125b",
                "modified_at": "2020-10-01T11:00:54.535Z",
                "name": "scikit_diabetes_model",
                "owner": "1000330999",
                "rev": "1",
                "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a"
            }
        }
    ]
}


### Create second model revision

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create_revision" 
target="_blank" rel="noopener no referrer">Create model revision</a>

In [34]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Updated model."}'
echo $REVISION_PAYLOAD | python -m json.tool

In [35]:
%env REVISION_PAYLOAD=$revision_payload

env: REVISION_PAYLOAD={
    "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
    "commit_message": "Updated model."
}


In [36]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "software_spec": {
            "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
            "name": "scikit-learn_0.20-py3.6"
        },
        "type": "scikit-learn_0.20"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Updated model.",
            "committed_at": "2020-10-01T11:18:46.002Z"
        },
        "created_at": "2020-10-01T11:00:54.535Z",
        "description": "This is description",
        "id": "c8c520cb-7298-4d30-a338-db62cdda125b",
        "modified_at": "2020-10-01T11:12:35.080Z",
        "name": "scikit_diabetes_model",
        "owner": "1000330999",
        "rev": "2",
        "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a"
    }
}


### Update model metadata  
  
For example update  model name or description

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_update" 
target="_blank" rel="noopener no referrer">Patch model</a>

In [37]:
%%bash --out update_payload

echo '[{"op": "add", "path": "/name", "value": "updated scikit model"}]' | python -m json.tool

In [38]:
%env UPDATE_PAYLOAD=$update_payload

env: UPDATE_PAYLOAD=[
    {
        "op": "add",
        "path": "/name",
        "value": "updated scikit model"
    }
]


In [39]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$UPDATE_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "software_spec": {
            "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
            "name": "scikit-learn_0.20-py3.6"
        },
        "type": "scikit-learn_0.20"
    },
    "metadata": {
        "commit_info": {
            "committed_at": "2020-10-01T11:18:56.048Z"
        },
        "created_at": "2020-10-01T11:00:54.535Z",
        "description": "This is description",
        "id": "c8c520cb-7298-4d30-a338-db62cdda125b",
        "modified_at": "2020-10-01T11:18:56.010Z",
        "name": "updated scikit model",
        "owner": "1000330999",
        "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a"
    }
}


### Upload new model content

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_upload_content" 
target="_blank" rel="noopener no referrer">Upload model content</a>

In [40]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@../../../../../../models/scikit/diabetes/model/new_diabetes_model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | python -m json.tool

{
    "attachment_id": "6485f528-fd65-47fd-9acc-800c6fae29c4",
    "content_format": "native",
    "persisted": true
}


#### List model revisions to see a new one just created

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_list_revisions" 
target="_blank" rel="noopener no referrer">List revisions</a>

In [41]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "first": {
        "href": "/ml/v4/models/c8c520cb-7298-4d30-a338-db62cdda125b/revisions?space_id=2219f598-6194-4e9a-9287-0730c69b5f4a&limit=100"
    },
    "limit": 100,
    "resources": [
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Updated model.",
                    "committed_at": "2020-10-01T11:18:46.002Z"
                },
                "created_at": "2020-10-01T11:00:54.535Z",
                "description": "This is description",
                "id": "c8c520cb-7298-4d30-a338-db62cdda125b",
                "modified_at": "2020-10-01T11:12:35.080Z",
                "name": "scikit_diabetes_model",
                "owner": "1000330999",
                "rev": "2",
                "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a"
            }
        },
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Initial model.",
                    "committed_

Now we have updated model content and model name in repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_get" 
target="_blank" rel="noopener no referrer">Get model details</a>

In [42]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "software_spec": {
            "id": "09c5a1d0-9c1e-4473-a344-eb7b665ff687",
            "name": "scikit-learn_0.20-py3.6"
        },
        "type": "scikit-learn_0.20"
    },
    "metadata": {
        "commit_info": {
            "committed_at": "2020-10-01T11:19:31.192Z"
        },
        "created_at": "2020-10-01T11:00:54.535Z",
        "description": "This is description",
        "id": "c8c520cb-7298-4d30-a338-db62cdda125b",
        "modified_at": "2020-10-01T11:19:05.671Z",
        "name": "updated scikit model",
        "owner": "1000330999",
        "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a"
    }
}


<a id="redeploy"></a>
## 5. Redeploy and score new version of the model

Below you can see how deployment can be updated with new version of the model without any change for scoring url.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_update" 
target="_blank" rel="noopener no referrer">Deployment update</a>

In [43]:
%%bash --out redeploy_payload

echo '[{"op": "replace", "path": "/asset", "value": {"id": "'"$MODEL_ID"'"}}]' \
     | python -m json.tool

In [44]:
%env REDEPLOY_PAYLOAD=$redeploy_payload

env: REDEPLOY_PAYLOAD=[
    {
        "op": "replace",
        "path": "/asset",
        "value": {
            "id": "c8c520cb-7298-4d30-a338-db62cdda125b"
        }
    }
]


In [45]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REDEPLOY_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "c8c520cb-7298-4d30-a338-db62cdda125b"
        },
        "custom": {},
        "deployed_asset_type": "model",
        "description": "This is description",
        "hardware_spec": {
            "id": "Not_Applicable",
            "name": "S",
            "num_nodes": 1
        },
        "name": "Diabetes deployment",
        "online": {},
        "space_id": "2219f598-6194-4e9a-9287-0730c69b5f4a",
        "status": {
            "online_url": {
                "url": "https://sep29b93-cpd-sep29b93.apps.ocwmlfvt454.cp.fyre.ibm.com/ml/v4/deployments/fda706ed-6eb7-476e-bbee-9f5a9e688aa0/predictions"
            },
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2020-10-01T11:13:04.822Z",
        "description": "This is description",
        "id": "fda706ed-6eb7-476e-bbee-9f5a9e688aa0",
        "modified_at": "2020-10-01T11:19:39.681Z",
        "name": "Diabetes deployment",
        "owner": "

### Score updated webservice

In [46]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2020-08-01" \
    | python -m json.tool

{
    "predictions": [
        {
            "fields": [
                "prediction"
            ],
            "values": [
                [
                    68.44064739029959
                ],
                [
                    178.20406005411206
                ]
            ]
        }
    ]
}


<a id="cleaning"></a>
## 6. Cleaning section

Below section is useful when you want to clean all of your previous work within this notebook.
Just convert below cells into the `code` and run them.

<a id="deployment_delete"></a>
### Deleting deployment
**Tip:** You can delete existing deployment by calling DELETE method.

<a id="model_delete"></a>
### Delete model from repository
**Tip:** If you want to completely remove your stored model and model metadata, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_delete" 
target="_blank" rel="noopener no referrer">Delete model from repository</a>

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `cURL` calls to store, deploy and score a scikit-learn ML model in WML. 


### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM  
**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.